In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
import joblib
from google.colab import drive

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
root_dir = '/content/drive/MyDrive/Colab Notebooks/'
dataset_path = os.path.join(root_dir,'Dataset256')

train_csv = os.path.join(root_dir,'Dataset_x256.csv')
test_csv = os.path.join(root_dir,'Steganography/validation_dataset.csv')
train_folder = os.path.join(dataset_path,'train')
validation_folder = os.path.join(dataset_path,'Valid')

In [ ]:
cover = '/content/drive/MyDrive/Colab Notebooks/Steganography/Lion.jpg'
secret = '/content/drive/MyDrive/Colab Notebooks/Steganography/Nikesh.jpg'

dataset = [{
        'cover_image':cover,
        'secret_image':secret
    }]
dataframe = pd.DataFrame(dataset)
cover = Image.open(cover)
secret = Image.open(secret)


In [ ]:

IMG_SIZE = 256
LEARNING_RATE  = 0.001
COVER_LOSS_WEIGHT = 1
SECRET_LOSS_WEIGHT = 1
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 1
EPOCHS = 1000
DECODER_LOSS_WEIGHT = 1

In [ ]:
imagetransformation = {
    'train_transforms':torchvision.transforms.Compose([torchvision.transforms.ToTensor()]),
    'valid_transforms':torchvision.transforms.Compose([torchvision.transforms.ToTensor()]),
    # 'test_transforms':torchvision.transforms.Compose([torchvision.transforms.Resize(cover.size),torchvision.transforms.ToTensor()])
}

In [ ]:
#conversion of image into resized format and return of trnasformed image
class SteganoDataset(torch.utils.data.Dataset):
    def __init__(self,dataset_csv,transforms,type_of_dataset,size='complete'):
        #self.dataset = pd.read_csv(dataset_csv)
        self.dataset = dataset_csv
        self.dataset = self.dataset.reset_index(drop=True)
        if size !='complete':
            self.dataset = self.dataset[:2]
        self.transforms = transforms
        self.type = type_of_dataset
    
    def __getitem__(self,index):
        cover_image = self.dataset.iloc[index]['cover_image']
        secret_image = self.dataset.iloc[index]['secret_image']
        cover_image = Image.open(os.path.join(dataset_path,'train',cover_image))
        secret_image = Image.open(os.path.join(dataset_path,'train',secret_image))
        transformed_cover_image = self.transforms(cover_image)
        transformed_secret_image = self.transforms(secret_image)
        return {
                'cover_image':transformed_cover_image,
                'secret_image':transformed_secret_image,
            }
        
    
    def __len__(self):
      return len(self.dataset)

In [ ]:
class PrepNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=3,out_channels=20,kernel_size=(3,3),stride=1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=3,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv4 = nn.Conv2d(in_channels=35,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv5 = nn.Conv2d(in_channels=35,out_channels=20,kernel_size=(3,3),stride=1,padding=1)
        self.conv6 = nn.Conv2d(in_channels=35,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
    
    def forward(self,secret_image):
        output_1 = F.relu(self.conv1(secret_image))
        output_2 = F.relu(self.conv2(secret_image))
        output_3 = F.relu(self.conv3(secret_image))
        
        concatenated_image = torch.cat([output_1,output_2,output_3],dim=1)
        output_4 = F.relu(self.conv4(concatenated_image))
        output_5 = F.relu(self.conv5(concatenated_image))
        output_6 = F.relu(self.conv6(concatenated_image))
        
        final_concat_image = torch.cat([output_4,output_5,output_6],dim=1)
        return final_concat_image

class HidingNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=38,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=38,out_channels=20,kernel_size=(3,3),stride=1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=38,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv4 = nn.Conv2d(in_channels=35,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv5 = nn.Conv2d(in_channels=35,out_channels=20,kernel_size=(3,3),stride=1,padding=1)
        self.conv6 = nn.Conv2d(in_channels=35,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv7 = nn.Conv2d(in_channels=35,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv8 = nn.Conv2d(in_channels=35,out_channels=20,kernel_size=(3,3),stride=1,padding=1)
        self.conv9 = nn.Conv2d(in_channels=35,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.final_layer = nn.Conv2d(in_channels=35,out_channels=3,kernel_size=(3,3),stride=1,padding=1)
        
    def forward(self,secret_image,cover_image):
        concatenated_secrets = torch.cat([cover_image,secret_image],dim=1)
        
        output_1 = F.relu(self.conv1(concatenated_secrets))
        output_2 = F.relu(self.conv2(concatenated_secrets))
        output_3 = F.relu(self.conv3(concatenated_secrets))
        concat_1 = torch.cat([output_1,output_2,output_3],dim=1)
        
        output_4 = F.relu(self.conv4(concat_1))
        output_5 = F.relu(self.conv5(concat_1))
        output_6 = F.relu(self.conv6(concat_1))
        concat_2 = torch.cat([output_4,output_5,output_6],dim=1)
        
        output_7 = F.relu(self.conv7(concat_2))
        output_8 = F.relu(self.conv8(concat_2))
        output_9 = F.relu(self.conv9(concat_2))
        concat_3 = torch.cat([output_7,output_8,output_9],dim=1)
        output_converted_image = F.relu(self.final_layer(concat_3))
        
        return output_converted_image

In [ ]:
class Encoder(nn.Module):
    def __init__(self,prep_network,hiding_network):
        super().__init__()
        self.prep_network = prep_network
        self.hiding_network = hiding_network
    
    def forward(self,cover_image,secret_image):
        encoded_secret_image = self.prep_network(secret_image)
        
        hidden_image = self.hiding_network(encoded_secret_image,
                                           cover_image
                                          )
#         hidden_image = (0.01**0.5)*torch.randn(hidden_image.size(),device=device)
        return hidden_image

In [ ]:
class RevealNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=3,out_channels=20,kernel_size=(3,3),stride=1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=3,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv4 = nn.Conv2d(in_channels=35,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv5 = nn.Conv2d(in_channels=35,out_channels=20,kernel_size=(3,3),stride=1,padding=1)
        self.conv6 = nn.Conv2d(in_channels=35,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        self.conv7 = nn.Conv2d(in_channels=35,out_channels=10,kernel_size=(3,3),stride=1,padding=1)
        self.conv8 = nn.Conv2d(in_channels=35,out_channels=20,kernel_size=(3,3),stride=1,padding=1)
        self.conv9 = nn.Conv2d(in_channels=35,out_channels=5,kernel_size=(5,5),stride=1,padding=2)
        
        
        self.final_layer = nn.Conv2d(in_channels=35,out_channels=3,kernel_size=(3,3),stride=1,padding=1)    
    
    def forward(self,hidden_image):
        
        output_1 = F.relu(self.conv1(hidden_image))
        output_2 = F.relu(self.conv2(hidden_image))
        output_3 = F.relu(self.conv3(hidden_image))
        concat_1 = torch.cat([output_1,output_2,output_3],dim=1)
        
        output_4 = F.relu(self.conv4(concat_1))
        output_5 = F.relu(self.conv5(concat_1))
        output_6 = F.relu(self.conv6(concat_1))
        concat_2 = torch.cat([output_4,output_5,output_6],dim=1)
        
        output_7 = F.relu(self.conv7(concat_2))
        output_8 = F.relu(self.conv8(concat_2))
        output_9 = F.relu(self.conv9(concat_2))
        concat_3 = torch.cat([output_7,output_8,output_9],dim=1)
        
        
        output_revealed_image = F.relu(self.final_layer(concat_3))
        
        return output_revealed_image

In [ ]:
class Decoder(nn.Module):
    def __init__(self,reveal_network):
        super().__init__()
        self.reveal_network = reveal_network
    
    def forward(self,hidden_image):
        reveal_image = self.reveal_network(hidden_image)
        return reveal_image

In [ ]:
class SteganoModel(nn.Module):
    def __init__(self,encoder,decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self,cover_image,secret_image,hidden_image,mode):
        if mode == 'full':
            for param in self.encoder.parameters():
                param.requires_grad = True
            for param in self.decoder.parameters():
                param.requires_grad = False
            hidden_image = self.encoder(cover_image,secret_image)
            reveal_image = self.decoder(hidden_image)
            return hidden_image,reveal_image
        elif mode == 'encoder':
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.decoder.parameters():
                param.requires_grad = False
            hidden_image = self.encoder(cover_image,secret_image)
            return hidden_image
        elif mode == 'decoder':
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.decoder.parameters():
                param.requires_grad = True
            
            reveal_image = self.decoder(hidden_image)
            return reveal_image

In [ ]:

# filename = os.path.join("/content/drive/MyDrive/Colab_Notebooks/Steganography/Completed_model2.joblib")
# # loaded_model = joblib.load(filename)
# joblib.dump(model, filename)

In [ ]:
prep_net = PrepNetwork()
hiding_network = HidingNetwork()

encoder = Encoder(prep_net,hiding_network)

reveal_net = RevealNetwork()


decoder = Decoder(reveal_net)

model = SteganoModel(encoder,decoder)
model.to(device)

In [ ]:
class SteganoLoss(nn.Module):
    def __init__(self,cover_weight,secret_weight):
        super().__init__()
        self.cover_weight = cover_weight
        self.secret_weight = secret_weight
        
    def forward(self,predicted_cover_image,cover_image,
               predicted_secret_image_1,secret_image_1):
    
        cover_loss = self.cover_weight*(F.mse_loss(predicted_cover_image,cover_image))
        secret_loss = self.secret_weight*(F.mse_loss(predicted_secret_image_1,secret_image_1)) 
        return cover_loss + secret_loss

class DecoderLoss(nn.Module):
    def __init__(self,decoder_loss_weight):
        super().__init__()
        self.decoder_loss_weight = decoder_loss_weight
    
    def forward(self,reveal_output,secret_image):
        reveal = self.decoder_loss_weight*F.mse_loss(reveal_output,secret_image)
        return reveal

In [ ]:

# training_csv_path = os.path.join(dataset_path,train_csv)
# test_csv_path = os.path.join(dataset_path,test_csv)

# training_dataset = SteganoDataset(training_csv_path,imagetransformation['train_transforms'],'train','complete')
# test_dataset = SteganoDataset(training_csv_path,imagetransformation['train_transforms'],'Train','complete')


In [ ]:
# test_dataset = SteganoDataset(dataframe,imagetransformation['test_transforms'],'Valid')

training_dataset = SteganoDataset(train_csv,imagetransformation['train_transforms'],'train','complete')
train_data_loader = torch.utils.data.DataLoader(training_dataset, 
                                                batch_size = TRAIN_BATCH_SIZE, 
                                                shuffle=True,
                                               drop_last = True,
                                               num_workers = 0
                                               )
# valid_data_loader = torch.utils.data.DataLoader(valid_dataset, 
#                                                 batch_size = VALID_BATCH_SIZE, 
#                                                 shuffle=True,
#                                                 drop_last = True,
#                                                 num_workers = 0
#                                                )
                                               
# test_data_loader = torch.utils.data.DataLoader(test_dataset, 
#                                                 batch_size = VALID_BATCH_SIZE, 
#                                                 shuffle=True,
#                                                 drop_last = True,
#                                                 num_workers = 0

In [ ]:

full_model_criterion = SteganoLoss(SECRET_LOSS_WEIGHT,COVER_LOSS_WEIGHT)

full_model_optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)

decoder_criterion = DecoderLoss(DECODER_LOSS_WEIGHT)
training_full_model_loss_list = []
decoder_loss_list = []


In [ ]:

def train(model,epochs,decoder_criterion,full_model_optimizer,full_model_criterion,learning_rate,training_iterator):
    print_every=1
    for epoch in range(epochs):
        for index,training_dict in enumerate(training_iterator):

            cover_image = training_dict['cover_image']
            cover_image = cover_image.to(device)

            secret_image = training_dict['secret_image']
            secret_image = secret_image.to(device)

            
            full_model_optimizer.zero_grad()

            encoder_output = model(cover_image,secret_image,secret_image,'encoder')

            hidden_image,reveal_image = model(cover_image,secret_image,secret_image,
                                                                'full')

            full_model_loss = full_model_criterion(hidden_image,cover_image,
                             reveal_image,secret_image
                            )
            full_model_loss.backward()
            full_model_optimizer.step()

            full_model_optimizer.zero_grad()
            reveal_output = model(cover_image,secret_image,
                                                                 encoder_output,'decoder')
            decoder_loss = decoder_criterion(reveal_output,secret_image)

            decoder_loss.backward()
            full_model_optimizer.step()

        training_full_model_loss_list.append(full_model_loss)
        decoder_loss_list.append(decoder_loss)
        if epoch % print_every == 0:
            print("Training full model loss at {} epochs is: {}".format(epoch, full_model_loss))
            print("Training decoder loss at {} epochs is: {}".format(epoch, decoder_loss))

    return model, training_full_model_loss_list,decoder_loss_list


Training full model loss at 0 epochs is: 0.14862200617790222
Training decoder loss at 0 epochs is: 0.056159209460020065
Training full model loss at 50 epochs is: 0.010316253639757633
Training decoder loss at 50 epochs is: 0.006905762478709221
Training full model loss at 100 epochs is: 0.005915718153119087
Training decoder loss at 100 epochs is: 0.004135390743613243
Training full model loss at 150 epochs is: 0.00597141869366169
Training decoder loss at 150 epochs is: 0.003256219904869795
Training full model loss at 200 epochs is: 0.004562869668006897
Training decoder loss at 200 epochs is: 0.0032390698324888945
Training full model loss at 250 epochs is: 0.004033954814076424
Training decoder loss at 250 epochs is: 0.00324784847907722
Training full model loss at 300 epochs is: 0.0028372027445584536
Training decoder loss at 300 epochs is: 0.002142705488950014
Training full model loss at 350 epochs is: 0.003511040238663554
Training decoder loss at 350 epochs is: 0.0028315880335867405
Training full model loss at 400 epochs is: 0.006889783777296543
Training decoder loss at 400 epochs is: 0.006818610243499279
Training full model loss at 450 epochs is: 0.0030810432508587837
Training decoder loss at 450 epochs is: 0.0027422530110925436
Training full model loss at 500 epochs is: 0.0026186055038124323
Training decoder loss at 500 epochs is: 0.00222468632273376
Training full model loss at 550 epochs is: 0.0022935508750379086
Training decoder loss at 550 epochs is: 0.0016986110713332891
Training full model loss at 600 epochs is: 0.0017199815483763814
Training decoder loss at 600 epochs is: 0.0012137810699641705
Training full model loss at 650 epochs is: 0.002981864381581545
Training decoder loss at 650 epochs is: 0.002139737131074071
Training full model loss at 700 epochs is: 0.00157183688133955
Training decoder loss at 700 epochs is: 0.0011825517285615206
Training full model loss at 750 epochs is: 0.001144192647188902
Training decoder loss at 750 epochs is: 0.0006921886233612895
Training full model loss at 800 epochs is: 0.0016071755671873689
Training decoder loss at 800 epochs is: 0.0011148437624797225
Training full model loss at 850 epochs is: 0.0016389719676226377
Training decoder loss at 850 epochs is: 0.0010738647542893887
Training full model loss at 900 epochs is: 0.0011910195462405682
Training decoder loss at 900 epochs is: 0.0007727841730229557
Training full model loss at 950 epochs is: 0.0011191064259037375
Training decoder loss at 950 epochs is: 0.0007212755153886974

In [ ]:
# model, training_full_model_loss_list, decoder_loss_list = train(model, EPOCHS,decoder_criterion, full_model_optimizer, full_model_criterion, LEARNING_RATE, train_data_loader)

In [ ]:
# filename = os.path.join("/content/drive/MyDrive/Colab Notebooks/Steganography/stegomodeltest.pt")
# # loaded_model = joblib.load(filename)
# torch.save(model,filename)

In [ ]:
test = torch.load('/content/drive/MyDrive/Colab Notebooks/Steganography/stegomodeltest.pkl')

In [ ]:
def predict(model,iterator,mode):

    predict_dict = next(iter(iterator))
    cover_image = predict_dict['cover_image']
    print(cover_image)
    cover_image = cover_image.to(device)
    secret_image_1 = predict_dict['secret_image']
    secret_image_1 = secret_image_1.to(device)
    
    model.eval()
    if mode =='decoder':
      reveal_image_1= model(cover_image,cover_image,cover_image,mode)
      # dot_graph = torchviz.make_dot(model(cover_image,cover_image,cover_image,mode))
      # dot_graph.render("decoder.dot")
    elif mode =='encoder':
      hidden_image= model(cover_image,secret_image_1,secret_image_1,mode)
      # dot_graph = torchviz.make_dot(model(cover_image,secret_image_1,secret_image_1,mode))
      # dot_graph.render("encoder.dot")
    elif mode=="full":
      hidden_image,reveal_image_1= model(cover_image,secret_image_1,secret_image_1,mode)
      # dot_graph = torchviz.make_dot(model(cover_image,secret_image_1,secret_image_1,mode))
      # dot_graph.render("full.dot")
      cover_image = cover_image * 255
      cover_image = cover_image.to(torch.device('cpu'))
      cover_image = cover_image.detach().to(torch.long)
      secret_image_1 = secret_image_1 * 255
      secret_image_1 = secret_image_1.to(torch.device('cpu'))
      secret_image_1 = secret_image_1.detach().to(torch.long)
    if mode =='encoder' or mode == 'full':
      hidden_image[hidden_image>1] = 1
      hidden_image = hidden_image * 255
      hidden_image = hidden_image.to(torch.device('cpu'))
      hidden_image = hidden_image.detach().to(torch.long)
      h = hidden_image[0].permute(1,2,0).numpy()
      im = Image.fromarray(h.astype(np.uint8))
      im.save("/content/drive/MyDrive/Colab Notebooks/stego_img.png")
      if mode == 'encoder': return h
    if mode =='decoder' or mode == 'full':
      reveal_image_1[reveal_image_1>1] = 1
      reveal_image_1 = reveal_image_1 * 255
      reveal_image_1 = reveal_image_1.to(torch.device('cpu'))
      reveal_image_1 = reveal_image_1.detach().to(torch.long)
      r = reveal_image_1[0].permute(1,2,0).numpy()
      im = Image.fromarray(r.astype(np.uint8))
      im.save("/content/drive/MyDrive/Colab Notebooks/revealed.png")
      if mode == 'encoder': return r
    return {
        'cover_image_grid':cover_image[0].permute(1,2,0).numpy(),
        'secret_image_1_grid':secret_image_1[0].permute(1,2,0).numpy(),
        'hidden_image_grid':h,
        'reveal_image_1_grid':r,
    }

In [ ]:
grids = predict(test,test_data_loader,'full')

In [ ]:
def plot(grids):
    plt.figure(figsize=(15,8))
    
    plt.subplot(241)
    plt.title('Cover Image')
    plt.imshow(grids['cover_image_grid'])

    plt.subplot(242)
    plt.title('Secret Image')
    plt.imshow(grids['secret_image_1_grid'])

    plt.subplot(245)
    plt.title('Hidden Image')
    plt.imshow(grids['hidden_image_grid'])
    
    plt.subplot(246)
    plt.title('Reveal Image')
    plt.imshow(grids['reveal_image_1_grid'])

    plt.savefig('/content/drive/MyDrive/Colab Notebooks/Plot.png')
    plt.show()
    

In [ ]:
'''
data = next(iter(test_data_loader))

cover_image = data['cover_image']
cover_image = cover_image.to(device)

secret_image_1 = data['secret_image']
secret_image_1 = secret_image_1.to(device)

model.eval()
    
hidden_image,reveal_image_1 = model(cover_image,secret_image_1,secret_image_1,'full')
'''

In [ ]:
#make_dot(hidden_image).render('hidden_image',format='png')

In [ ]:
#make_dot(reveal_image_1).render('reveal_image',format='png')

In [ ]:

def plot_loss(loss_list,num):
    plt.plot(torch.tensor(loss_list, device = 'cpu'))
    if(num==1):
      plt.title('full mode loss')
      plt.xlabel("epochs")
      plt.ylabel("loss")
      plt.savefig('/content/drive/MyDrive/Colab Notebooks/fullmodeloss11.png')
      plt.show()
    else:
      plt.title('decoder loss')
      plt.xlabel("epochs")
      plt.ylabel("loss")
      plt.savefig('/content/drive/MyDrive/Colab Notebooks/decoderloss11.png')
      plt.show()


In [ ]:
plot(grids)

<h1>Analysis Code</h1>

In [ ]:
import math
import numpy as np
import cv2
from matplotlib.image import imread
from scipy.stats import entropy
import matplotlib.pyplot as plt

def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def snr(stego_image, original_image):
    # Calculate the noise image
    noise_image = stego_image - original_image
    # Calculate the average noise power
    noise_power = np.mean(noise_image ** 2)
    # Calculate the average signal power
    signal_power = np.mean(original_image ** 2)
    # Calculate the SNR
    snr = signal_power / noise_power
    # Convert the SNR to decibels
    snr_db = 10 * np.log10(snr)
    return snr_db

def ssim(image1, image2):
    # Convert the images to grayscale
    image1_gray = cv2.cvtColor(cv2.convertScaleAbs(image1), cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(cv2.convertScaleAbs(image2), cv2.COLOR_BGR2GRAY)
    # Calculate the SSIM
    ssim, _ = cv2.quality.QualityMSE_compute(image1_gray, image2_gray)
    return ssim

def correlation_coefficient(image1, image2):
    # Flatten the images into 1D arrays
    image1_flat = image1.flatten()
    image2_flat = image2.flatten()
    # Calculate the correlation coefficient
    correlation = np.corrcoef(image1_flat, image2_flat)[0, 1]
    return correlation

def image_entropy(image):
    # Flatten the image into a 1D array
    image_flat = image.flatten()
    # Calculate the entropy
    return entropy(image_flat)

def pixel_frequencies(image):
    # Flatten the image into a 1D array
    image_flat = image.flatten()
    # Calculate the frequencies of each pixel value
    unique, counts = np.unique(image_flat, return_counts=True)
    frequencies = counts / len(image_flat)
    return unique, frequencies,counts

def plot_frequencies(unique, frequencies,counts):
    # Plot the frequencies
    plt.plot(unique, counts)
    plt.xlim([0, 256])
    plt.show()

<h1>For Loop</h1>

In [ ]:
cover = Image.open(cover)
secret = Image.open(secret)
grids = predict(model,test_data_loader,'full')

In [ ]:
cpath = '/content/drive/MyDrive/Colab Notebooks/Steganography/Dataset/cover'
spath = '/content/drive/MyDrive/Colab Notebooks/Steganography/Dataset/train'
listofcimg = os.listdir(cpath)
listofsimg = os.listdir(spath)

In [ ]:
df = pd.DataFrame(columns=["psnrC", "psnrS", "snrC", "snrS", "ssimC", "ssimS", "corrC", "corrS", "entC", "entH", "entS", "entR"])
for i in range(1000):
  dataset = [{
        'cover_image':os.path.join(cpath,listofcimg[i]),
        'secret_image':os.path.join(spath,listofsimg[i])
    }]
  dataframe = pd.DataFrame(dataset)
  test_dataset = SteganoDataset(dataframe,imagetransformation['test_transforms'],'Valid')                                               
  test_data_loader = torch.utils.data.DataLoader(test_dataset, 
                                                  batch_size = VALID_BATCH_SIZE, 
                                                  shuffle=True,
                                                  drop_last = True,
                                                  num_workers = 0
                                                )

  grids = predict(model,test_data_loader,'full')
  psnrC = psnr(grids['cover_image_grid'],grids['hidden_image_grid'])
  psnrS = psnr(grids['secret_image_1_grid'],grids['reveal_image_1_grid'])
  snrC = snr(grids['cover_image_grid'],grids['hidden_image_grid'])
  snrS = snr(grids['secret_image_1_grid'],grids['reveal_image_1_grid'])
  ssimC = ssim(grids['cover_image_grid'],grids['hidden_image_grid'])[0]
  ssimS = ssim(grids['secret_image_1_grid'],grids['reveal_image_1_grid'])[0]
  corrC = correlation_coefficient(grids['cover_image_grid'],grids['hidden_image_grid'])
  corrS = correlation_coefficient(grids['secret_image_1_grid'],grids['reveal_image_1_grid'])
  entC = image_entropy(grids['cover_image_grid'])
  entH = image_entropy(grids['hidden_image_grid'])
  entS = image_entropy(grids['secret_image_1_grid'])
  entR = image_entropy(grids['reveal_image_1_grid'])
  df = df.append({"psnrC": psnrC, "psnrS": psnrS, "snrC": snrC, "snrS": snrS, "ssimC": ssimC, "ssimS": ssimS, "corrC": corrC, "corrS": corrS, "entC": entC, "entH": entH, "entS": entS, "entR": entR}, ignore_index=True)

df.to_csv("/content/drive/MyDrive/Colab Notebooks/Steganography/table.csv", index=False)

In [ ]:
df_norm = df.apply(lambda x: (x - x.mean()) / x.std())

In [ ]:
df.plot(y='psnrS')
plt.xlabel("Image number")
plt.ylabel("PSNR")
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.mean(axis=0)

In [ ]:
from skimage.metrics import structural_similarity as ssim

In [ ]:
ssimValue = ssim(grids['cover_image_grid'],grids['secret_image_1_grid'],win_size=11, multichannel = True)

In [ ]:
ssimValue

In [ ]:
from pyssim import SSIM 
ssim = SSIM()
score = ssim.compute(grids['cover_image_grid'],grids['secret_image_1_grid'])

In [ ]:
!pip install pyssim